In [4]:
import pandas as pd
from sodapy import Socrata
import json
from geopy.distance import geodesic
import geopandas as gpd
from shapely.wkt import loads

In [8]:
jsonData = json.load(open('busRouteData.json'))
jsonData['data']
busRouteData = pd.DataFrame(jsonData['data'])

In [9]:
sa1Centroids = pd.read_excel("centroids.xlsx")

client = Socrata("www.data.act.gov.au", None)

results = client.get("3fd4-5fkk", limit=2000)


results_df = pd.DataFrame.from_records(results)
results_df = results_df.astype({'sa1':str,'school':str,'no_of_students':float})
results_df['concatenated_column'] = results_df['school'] + ' ' + results_df['sa1']
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   school               2000 non-null   object 
 1   sa1                  2000 non-null   object 
 2   no_of_students       2000 non-null   float64
 3   concatenated_column  2000 non-null   object 
dtypes: float64(1), object(3)
memory usage: 62.6+ KB


In [10]:
sa1Centroids

,SA1_CODE21,CHG_FLAG21,CHG_LBL21,SA2_CODE21,SA2_NAME21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,xcoord,ycoord
0,10102100701,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,362.8727,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.857708,-35.146090
1,10102100702,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,229.7459,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.803063,-35.434670
2,10102100703,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,2.3910,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.789593,-35.438611
3,10102100704,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,1.2816,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.802280,-35.438995
4,10102100705,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,1.1978,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.796809,-35.449957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68390,90104100408,0,No change,901041004,Norfolk Island,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,2.3801,http://linked.data.gov.au/dataset/asgsed3/SA1/...,167.952141,-29.120693
68391,90104100408,0,No change,901041004,Norfolk Island,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,2.3801,http://linked.data.gov.au/dataset/asgsed3/SA1/...,167.949564,-29.113051
68392,99797979993,0,No change,997979799,Migratory - Offshore - Shipping (OT),99797,Migratory - Offshore - Shipping (OT),997,Migratory - Offshore - Shipping (OT),99799,Migratory - Offshore - Shipping (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA1/...,NaN,NaN
68393,99999949999,0,No change,999999499,No usual address (OT),99999,No usual address (OT),999,No usual address (OT),99499,No usual address (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA1/...,NaN,NaN


In [11]:
mean_no_of_students = results_df.drop(columns=['sa1','concatenated_column']).groupby(by="school").mean()
mean_no_of_students.sort_values(by='no_of_students',ascending=True)

,no_of_students
school,
The Canberra College,5.000000
Erindale College,5.578947
Hawker College,5.736842
Lake Tuggeranong College,6.096154
Belconnen High School,6.500000
...,...
Aranda Primary School,21.727273
Harrison School,22.208955
Amaroo School,24.030769


In [12]:
results_df[results_df['school'] == 'The Canberra College'].groupby(["school","concatenated_column"]).sum()

sa1  \
school               concatenated_column                             
The Canberra College The Canberra College 10102101150  10102101150   
                     The Canberra College 80107107606  80107107606   
                     The Canberra College 80107108204  80107108204   
                     The Canberra College 80107108207  80107108207   
                     The Canberra College 80107108227  80107108227   
                     The Canberra College 80108109105  80108109105   
                     The Canberra College 80108109202  80108109202   
                     The Canberra College 80108109204  80108109204   
                     The Canberra College 80108109206  80108109206   
                     The Canberra College 80108109306  80108109306   
                     The Canberra College 80108109405  80108109405   
                     The Canberra College 80108109601  80108109601   
                     The Canberra College 80108109807  80108109807   
                     The Canberra College 80108109809  80108109809   
                     The Canberra College 80109109902  80109109902   
                     The Canberra College 80109110008  80109110008   
                     The Canberra College 80109110304  80109110304   
                     The Canberra College 80109110401  80109110401   
                     The Canberra College 80109110502  80109110502   
                     The Canberra College 80109110504  80109110504   
                     The Canberra College 80109110507  80109110507   

                                                       no_of_students  
school               concatenated_column                               
The Canberra College The Canberra College 10102101150             5.0  
                     The Canberra College 80107107606             5.0  
                     The Canberra College 80107108204             5.0  
                     The Canberra College 80107108207             5.0  
                     The Canberra College 80107108227             5.0  
                     The Canberra College 80108109105             5.0  
                     The Canberra College 80108109202             5.0  
                     The Canberra College 80108109204             5.0  
                     The Canberra College 80108109206             5.0  
                     The Canberra College 80108109306             5.0  
                     The Canberra College 80108109405             5.0  
                     The Canberra College 80108109601             5.0  
                     The Canberra College 80108109807             5.0  
                     The Canberra College 80108109809             5.0  
                     The Canberra College 80109109902             5.0  
                     The Canberra College 80109110008             5.0  
                     The Canberra College 80109110304             5.0  
                     The Canberra College 80109110401             5.0  
                     The Canberra College 80109110502             5.0  
                     The Canberra College 80109110504             5.0  
                     The Canberra College 80109110507             5.0

In [13]:
results_df[results_df['concatenated_column'] == 'The Canberra College 10102101150']

,school,sa1,no_of_students,concatenated_column
1979,The Canberra College,10102101150,5.0,The Canberra College 10102101150


In [14]:
results_df[results_df['school'] == 'Garran Primary School'].groupby(["school","concatenated_column"]).sum()

sa1  \
school                concatenated_column                              
Garran Primary School Garran Primary School 80106106903  80106106903   
                      Garran Primary School 80108109304  80108109304   
                      Garran Primary School 80109110202  80109110202   
                      Garran Primary School 80109110203  80109110203   
                      Garran Primary School 80109110204  80109110204   
                      Garran Primary School 80109110205  80109110205   
                      Garran Primary School 80109110206  80109110206   
                      Garran Primary School 80109110207  80109110207   
                      Garran Primary School 80109110209  80109110209   
                      Garran Primary School 80109110211  80109110211   
                      Garran Primary School 80109110303  80109110303   
                      Garran Primary School 80109110307  80109110307   
                      Garran Primary School 80109110507  80109110507   
                      Garran Primary School 80109110701  80109110701   
                      Garran Primary School 80109110702  80109110702   
                      Garran Primary School 80109111001  80109111001   

                                                         no_of_students  
school                concatenated_column                                
Garran Primary School Garran Primary School 80106106903             7.0  
                      Garran Primary School 80108109304             6.0  
                      Garran Primary School 80109110202            65.0  
                      Garran Primary School 80109110203            44.0  
                      Garran Primary School 80109110204            10.0  
                      Garran Primary School 80109110205            23.0  
                      Garran Primary School 80109110206            21.0  
                      Garran Primary School 80109110207           128.0  
                      Garran Primary School 80109110209            25.0  
                      Garran Primary School 80109110211            56.0  
                      Garran Primary School 80109110303             5.0  
                      Garran Primary School 80109110307             9.0  
                      Garran Primary School 80109110507             5.0  
                      Garran Primary School 80109110701             5.0  
                      Garran Primary School 80109110702            18.0  
                      Garran Primary School 80109111001             5.0

In [15]:
results_df[results_df['concatenated_column'] == 'Garran Primary School 80109110207']

,school,sa1,no_of_students,concatenated_column
739,Garran Primary School,80109110207,128.0,Garran Primary School 80109110207


In [16]:
results_df.groupby(by="sa1").sum().sort_values(by='no_of_students',ascending=True)

,school,no_of_students,concatenated_column
sa1,,,
10102101216,Campbell High School,5.0,Campbell High School 10102101216
10102101211,Campbell Primary School,5.0,Campbell Primary School 10102101211
10106154208,Campbell High School,5.0,Campbell High School 10106154208
10102101222,Campbell High School,5.0,Campbell High School 10102101222
80109110901,Lyons Early Childhood School,5.0,Lyons Early Childhood School 80109110901
...,...,...,...
80104103516,Amaroo SchoolNeville Bonner Primary School,91.0,Amaroo School 80104103516Neville Bonner Primar...
80108109604,Alfred Deakin High SchoolChapman Primary Schoo...,97.0,Alfred Deakin High School 80108109604Chapman P...
80104103410,Amaroo SchoolGungahlin College,103.0,Amaroo School 80104103410Gungahlin College 801...


In [17]:
results_df[results_df['sa1'] == '80109110207']

,school,sa1,no_of_students,concatenated_column
71,Alfred Deakin High School,80109110207,14.0,Alfred Deakin High School 80109110207
739,Garran Primary School,80109110207,128.0,Garran Primary School 80109110207


In [18]:
def load_valid(geo):
    try:
        return loads(geo)
    except:
        return np.nan
    
busRouteData['routes'] = busRouteData[8].apply(load_valid)



busRouteGeoData = gpd.GeoDataFrame(busRouteData, geometry="routes")

In [19]:
busRouteGeoData['centroid'] = busRouteGeoData.routes.centroid
busRouteGeoData['centroid_x'] = busRouteGeoData.routes.centroid.x
busRouteGeoData['centroid_y'] = busRouteGeoData.routes.centroid.y

In [20]:
busRouteGeoData

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,routes,centroid,centroid_x,centroid_y
0,row-di3a_ghjk-3zsr,00000000-0000-0000-544B-608950EF1E53,0,1589437070,None,1589437070,None,{ },MULTILINESTRING ((149.12561599999998 -35.27913...,0,...,None,3,0042AC,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.12562 -35.27913, 149.125...",POINT (149.08057 -35.30856),149.080567,-35.308561
1,row-hayc.igqu~sn92,00000000-0000-0000-1A78-9086FDC5DFC4,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.022007 -35.299196, 149.0...",0,...,None,3,0042AC,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.02201 -35.2992, 149.0220...",POINT (149.08051 -35.30814),149.080513,-35.308143
2,row-3enu.suti.miak,00000000-0000-0000-0C0B-D7CC64C0D90D,0,1589437070,None,1589437070,None,{ },MULTILINESTRING ((149.12561599999998 -35.27913...,0,...,None,3,0042AC,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.12562 -35.27913, 149.125...",POINT (149.08606 -35.37746),149.086061,-35.377460
3,row-dmcp_7vw8_qqyb,00000000-0000-0000-A2C5-C45DAC6CD1C3,0,1589437070,None,1589437070,None,{ },MULTILINESTRING ((149.09249400000002 -35.45621...,0,...,None,3,0042AC,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.09249 -35.45621, 149.092...",POINT (149.0856 -35.37804),149.085599,-35.378036
4,row-uy72~zytr-naxh,00000000-0000-0000-F15B-C2EAF40CC732,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.135127 -35.185345, 149.1...",0,...,None,3,0042AC,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.13513 -35.18534, 149.135...",POINT (149.14859 -35.20687),149.148590,-35.206868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,row-nie4_devw-xs4w,00000000-0000-0000-31BE-B18BA71AF49A,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.134445 -35.25104, 149.13...",0,...,None,3,0042AC,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.13444 -35.25104, 149.133...",POINT (149.09934 -35.24335),149.099344,-35.243345
200,row-hkye_adk9.gpzv,00000000-0000-0000-A5AC-2C3147C51B9F,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.086966 -35.344586, 149.0...",0,...,None,3,0042AC,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.08697 -35.34459, 149.087...",POINT (149.13796 -35.34313),149.137960,-35.343127
201,row-iyci~geej-rkyz,00000000-0000-0000-A357-2DCCD81AB312,0,1589437070,None,1589437070,None,{ },MULTILINESTRING ((149.01948900000002 -35.22248...,0,...,None,3,0042AC,FFFFFF,None,Bus,"MULTILINESTRING ((149.01949 -35.22248, 149.019...",POINT (149.01018 -35.22937),149.010181,-35.229370
202,row-gmz2~32v2_9q9d,00000000-0000-0000-90D1-741561923EB3,0,1589437070,None,1589437070,None,{ },MULTILINESTRING ((149.01948900000002 -35.22248...,0,...,None,3,0042AC,FFFFFF,None,Bus,"MULTILINESTRING ((149.01949 -35.22248, 149.019...",POINT (149.01092 -35.22713),149.010924,-35.227131


In [21]:
sa1Centroids

,SA1_CODE21,CHG_FLAG21,CHG_LBL21,SA2_CODE21,SA2_NAME21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,xcoord,ycoord
0,10102100701,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,362.8727,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.857708,-35.146090
1,10102100702,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,229.7459,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.803063,-35.434670
2,10102100703,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,2.3910,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.789593,-35.438611
3,10102100704,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,1.2816,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.802280,-35.438995
4,10102100705,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,1.1978,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.796809,-35.449957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68390,90104100408,0,No change,901041004,Norfolk Island,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,2.3801,http://linked.data.gov.au/dataset/asgsed3/SA1/...,167.952141,-29.120693
68391,90104100408,0,No change,901041004,Norfolk Island,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,2.3801,http://linked.data.gov.au/dataset/asgsed3/SA1/...,167.949564,-29.113051
68392,99797979993,0,No change,997979799,Migratory - Offshore - Shipping (OT),99797,Migratory - Offshore - Shipping (OT),997,Migratory - Offshore - Shipping (OT),99799,Migratory - Offshore - Shipping (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA1/...,NaN,NaN
68393,99999949999,0,No change,999999499,No usual address (OT),99999,No usual address (OT),999,No usual address (OT),99499,No usual address (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA1/...,NaN,NaN


In [32]:
def calculateMinDistance(DataFrame,x,y):
    DataFrame["targetX"] = x
    DataFrame["targetY"] = y
    # Create GeoDataFrames from coordinates
    gdf_start = gpd.GeoDataFrame(DataFrame, geometry=gpd.points_from_xy(DataFrame.centroid_x, DataFrame.centroid_y), crs="EPSG:4326")
    gdf_end = gpd.GeoDataFrame(DataFrame, geometry=gpd.points_from_xy(DataFrame.targetX, DataFrame.targetY), crs="EPSG:4326")

    # Reproject to a projected CRS for accurate distance calculation (e.g., a suitable UTM zone)
    gdf_start_proj = gdf_start.to_crs("EPSG:3857") # Example: Web Mercator
    gdf_end_proj = gdf_end.to_crs("EPSG:3857")

    # Calculate distance
    DataFrame['geopandas_distance'] = gdf_start_proj.distance(gdf_end_proj)
    #print(DataFrame['geopandas_distance'].min()/1000)
    #print(DataFrame)
    return DataFrame['geopandas_distance'].min()/1000
        

float(calculateMinDistance(busRouteGeoData,sa1Centroids.xcoord,sa1Centroids.ycoord))

11.1048156877242

In [71]:
def calculateDistances(DataFrame,x,y):
    DataFrame["targetX"] = x
    DataFrame["targetY"] = y
    # Create GeoDataFrames from coordinates
    gdf_start = gpd.GeoDataFrame(DataFrame, geometry=gpd.points_from_xy(DataFrame.centroid_x, DataFrame.centroid_y), crs="EPSG:4326")
    gdf_end = gpd.GeoDataFrame(DataFrame, geometry=gpd.points_from_xy(DataFrame.targetX, DataFrame.targetY), crs="EPSG:4326")

    # Reproject to a projected CRS for accurate distance calculation (e.g., a suitable UTM zone)
    gdf_start_proj = gdf_start.to_crs("EPSG:3857") # Example: Web Mercator
    gdf_end_proj = gdf_end.to_crs("EPSG:3857")

    # Calculate distance
    DataFrame['geopandas_distance'] = gdf_start_proj.distance(gdf_end_proj)/1000
    #print(DataFrame['geopandas_distance'].min()/1000)
    #print(DataFrame)
    return DataFrame
        

calculateDistances(busRouteGeoData,float(sa1Centroids[sa1Centroids['SA1_CODE21'] == '80104103410'].xcoord),float(sa1Centroids[sa1Centroids['SA1_CODE21'] == '80104103410'].ycoord))

C:\Users\bsbal\AppData\Local\Temp/ipykernel_22388/1820138379.py:19: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  calculateDistances(busRouteGeoData,float(sa1Centroids[sa1Centroids['SA1_CODE21'] == '80104103410'].xcoord),float(sa1Centroids[sa1Centroids['SA1_CODE21'] == '80104103410'].ycoord))


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,routes,centroid,centroid_x,centroid_y,targetX,targetY,geopandas_distance
0,row-di3a_ghjk-3zsr,00000000-0000-0000-544B-608950EF1E53,0,1589437070,None,1589437070,None,{ },MULTILINESTRING ((149.12561599999998 -35.27913...,0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.12562 -35.27913, 149.125...",POINT (149.08057 -35.30856),149.080567,-35.308561,149.130306,-35.170043,19.674572
1,row-hayc.igqu~sn92,00000000-0000-0000-1A78-9086FDC5DFC4,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.022007 -35.299196, 149.0...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.02201 -35.2992, 149.0220...",POINT (149.08051 -35.30814),149.080513,-35.308143,149.130306,-35.170043,19.621550
2,row-3enu.suti.miak,00000000-0000-0000-0C0B-D7CC64C0D90D,0,1589437070,None,1589437070,None,{ },MULTILINESTRING ((149.12561599999998 -35.27913...,0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.12562 -35.27913, 149.125...",POINT (149.08606 -35.37746),149.086061,-35.377460,149.130306,-35.170043,28.707687
3,row-dmcp_7vw8_qqyb,00000000-0000-0000-A2C5-C45DAC6CD1C3,0,1589437070,None,1589437070,None,{ },MULTILINESTRING ((149.09249400000002 -35.45621...,0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.09249 -35.45621, 149.092...",POINT (149.0856 -35.37804),149.085599,-35.378036,149.130306,-35.170043,28.794011
4,row-uy72~zytr-naxh,00000000-0000-0000-F15B-C2EAF40CC732,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.135127 -35.185345, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.13513 -35.18534, 149.135...",POINT (149.14859 -35.20687),149.148590,-35.206868,149.130306,-35.170043,5.413080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,row-nie4_devw-xs4w,00000000-0000-0000-31BE-B18BA71AF49A,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.134445 -35.25104, 149.13...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.13444 -35.25104, 149.133...",POINT (149.09934 -35.24335),149.099344,-35.243345,149.130306,-35.170043,10.564781
200,row-hkye_adk9.gpzv,00000000-0000-0000-A5AC-2C3147C51B9F,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.086966 -35.344586, 149.0...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.08697 -35.34459, 149.087...",POINT (149.13796 -35.34313),149.137960,-35.343127,149.130306,-35.170043,23.611033
201,row-iyci~geej-rkyz,00000000-0000-0000-A357-2DCCD81AB312,0,1589437070,None,1589437070,None,{ },MULTILINESTRING ((149.01948900000002 -35.22248...,0,...,FFFFFF,None,Bus,"MULTILINESTRING ((149.01949 -35.22248, 149.019...",POINT (149.01018 -35.22937),149.010181,-35.229370,149.130306,-35.170043,15.624890
202,row-gmz2~32v2_9q9d,00000000-0000-0000-90D1-741561923EB3,0,1589437070,None,1589437070,None,{ },MULTILINESTRING ((149.01948900000002 -35.22248...,0,...,FFFFFF,None,Bus,"MULTILINESTRING ((149.01949 -35.22248, 149.019...",POINT (149.01092 -35.22713),149.010924,-35.227131,149.130306,-35.170043,15.397803


In [35]:
sa1Centroids['distanceKM'] = sa1Centroids.apply(lambda row: float(calculateMinDistance(busRouteGeoData,row.xcoord,row.ycoord)), axis=1)

In [38]:
sa1Centroids['sa1'] = sa1Centroids['SA1_CODE21']
sa1Centroids

,SA1_CODE21,CHG_FLAG21,CHG_LBL21,SA2_CODE21,SA2_NAME21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,...,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,xcoord,ycoord,distanceKM,sa1
0,10102100701,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,...,1,New South Wales,AUS,Australia,362.8727,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.857708,-35.146090,79.229954,10102100701
1,10102100702,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,...,1,New South Wales,AUS,Australia,229.7459,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.803063,-35.434670,74.211700,10102100702
2,10102100703,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,...,1,New South Wales,AUS,Australia,2.3910,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.789593,-35.438611,72.920561,10102100703
3,10102100704,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,...,1,New South Wales,AUS,Australia,1.2816,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.802280,-35.438995,74.290533,10102100704
4,10102100705,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,...,1,New South Wales,AUS,Australia,1.1978,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.796809,-35.449957,74.060324,10102100705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68390,90104100408,0,No change,901041004,Norfolk Island,90104,Norfolk Island,901,Other Territories,9OTER,...,9,Other Territories,AUS,Australia,2.3801,http://linked.data.gov.au/dataset/asgsed3/SA1/...,167.952141,-29.120693,2240.213735,90104100408
68391,90104100408,0,No change,901041004,Norfolk Island,90104,Norfolk Island,901,Other Territories,9OTER,...,9,Other Territories,AUS,Australia,2.3801,http://linked.data.gov.au/dataset/asgsed3/SA1/...,167.949564,-29.113051,2240.291708,90104100408
68392,99797979993,0,No change,997979799,Migratory - Offshore - Shipping (OT),99797,Migratory - Offshore - Shipping (OT),997,Migratory - Offshore - Shipping (OT),99799,...,9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA1/...,NaN,NaN,NaN,99797979993
68393,99999949999,0,No change,999999499,No usual address (OT),99999,No usual address (OT),999,No usual address (OT),99499,...,9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA1/...,NaN,NaN,NaN,99999949999


In [51]:
resultdf = pd.merge(results_df,sa1Centroids,on = 'sa1')
resultdf[(resultdf['distanceKM'] >= 4) & (resultdf['STE_NAME21'] != 'New South Wales')]

,school,sa1,no_of_students,concatenated_column,SA1_CODE21,CHG_FLAG21,CHG_LBL21,SA2_CODE21,SA2_NAME21,SA3_CODE21,...,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,xcoord,ycoord,distanceKM
302,Campbell Primary School,80103111401,8.0,Campbell Primary School 80103111401,80103111401,0,No change,801031114,Canberra Airport,80103,...,Australian Capital Territory,8,Australian Capital Territory,AUS,Australia,5.5374,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.196254,-35.304026,4.509125
722,Gold Creek School,80104104301,5.0,Gold Creek School 80104104301,80104104301,0,No change,801041043,Hall,80104,...,Australian Capital Territory,8,Australian Capital Territory,AUS,Australia,1.7392,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.067232,-35.170066,4.364618
731,Gold Creek School,80104104302,6.0,Gold Creek School 80104104302,80104104302,0,No change,801041043,Hall,80104,...,Australian Capital Territory,8,Australian Capital Territory,AUS,Australia,1.1326,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.055942,-35.167739,4.234336
1628,Narrabundah College,80103111301,6.0,Narrabundah College 80103111301,80103111301,0,No change,801031113,Canberra East,80103,...,Australian Capital Territory,8,Australian Capital Territory,AUS,Australia,0.4085,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.228156,-35.340048,9.013874
1846,Telopea Park School,80103111301,5.0,Telopea Park School 80103111301,80103111301,0,No change,801031113,Canberra East,80103,...,Australian Capital Territory,8,Australian Capital Territory,AUS,Australia,0.4085,http://linked.data.gov.au/dataset/asgsed3/SA1/...,149.228156,-35.340048,9.013874


In [69]:
sa1Centroids[sa1Centroids['SA1_CODE21'] == '80104103410'].xcoord

67362    149.130306
Name: xcoord, dtype: float64

In [72]:
test1 = calculateDistances(busRouteGeoData,float(sa1Centroids[sa1Centroids['SA1_CODE21'] == '80104103410'].xcoord),float(sa1Centroids[sa1Centroids['SA1_CODE21'] == '80104103410'].ycoord))
test1[test1['geopandas_distance'] <=5]

C:\Users\bsbal\AppData\Local\Temp/ipykernel_22388/2702074610.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  test1 = calculateDistances(busRouteGeoData,float(sa1Centroids[sa1Centroids['SA1_CODE21'] == '80104103410'].xcoord),float(sa1Centroids[sa1Centroids['SA1_CODE21'] == '80104103410'].ycoord))


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,routes,centroid,centroid_x,centroid_y,targetX,targetY,geopandas_distance
6,row-y68b-6ky4_yxn9,00000000-0000-0000-5FBD-65F16C8BBC8D,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.135127 -35.185345, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.13513 -35.18534, 149.135...",POINT (149.14933 -35.20251),149.149329,-35.202514,149.130306,-35.170043,4.903503
11,row-id35-shih~shjz,00000000-0000-0000-D1DC-99E0A6BDE37C,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.134996 -35.184618, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.135 -35.18462, 149.13505...",POINT (149.13703 -35.16923),149.137029,-35.169231,149.130306,-35.170043,0.756535
12,row-hnm8.89gx_jubk,00000000-0000-0000-76D4-034A416A537B,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.134996 -35.184618, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.135 -35.18462, 149.13505...",POINT (149.13697 -35.16965),149.136974,-35.169645,149.130306,-35.170043,0.744269
13,row-355y~yjis-s388,00000000-0000-0000-756A-72FFC31C62BB,0,1589437070,None,1589437070,None,{ },MULTILINESTRING ((149.12411799999998 -35.18698...,0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.12412 -35.18698, 149.124...",POINT (149.13644 -35.17102),149.136439,-35.171019,149.130306,-35.170043,0.695466
14,row-yh4w~ygqj~2mt6,00000000-0000-0000-2486-7BB7F9464708,0,1589437070,None,1589437070,None,{ },MULTILINESTRING ((149.12411799999998 -35.18698...,0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.12412 -35.18698, 149.124...",POINT (149.13644 -35.17102),149.136439,-35.171019,149.130306,-35.170043,0.695466
15,row-ytqk.np6i~ai8j,00000000-0000-0000-6CA6-B95B1861A1C2,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.134996 -35.184618, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.135 -35.18462, 149.13505...",POINT (149.13697 -35.16965),149.136974,-35.169645,149.130306,-35.170043,0.744269
16,row-mi8r~f4i3.xgsg,00000000-0000-0000-C987-418F6AC5A41B,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.134996 -35.184618, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.135 -35.18462, 149.13505...",POINT (149.13699 -35.16913),149.136985,-35.169128,149.130306,-35.170043,0.753896
17,row-hw3s~cksg_6j8n,00000000-0000-0000-02A9-275828C03DC9,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.134996 -35.184618, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.135 -35.18462, 149.13505...",POINT (149.13693 -35.16954),149.136932,-35.169543,149.130306,-35.170043,0.740697
18,row-cb6d-4hcw-p3t3,00000000-0000-0000-C067-69F0FACD6639,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.122171 -35.173339, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.12217 -35.17334, 149.122...",POINT (149.13863 -35.16737),149.138629,-35.167366,149.130306,-35.170043,0.995704
19,row-9dg4.uty4_h7v5,00000000-0000-0000-8370-2BE49E2DD338,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.143879 -35.170876, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.14388 -35.17088, 149.143...",POINT (149.13822 -35.18034),149.138216,-35.180344,149.130306,-35.170043,1.656269


In [74]:
test1 = calculateDistances(busRouteGeoData,float(sa1Centroids[sa1Centroids['SA1_CODE21'] == '80104111801'].xcoord),float(sa1Centroids[sa1Centroids['SA1_CODE21'] == '80104111801'].ycoord))
test1[test1['geopandas_distance'] <=5]

C:\Users\bsbal\AppData\Local\Temp/ipykernel_22388/3886653346.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  test1 = calculateDistances(busRouteGeoData,float(sa1Centroids[sa1Centroids['SA1_CODE21'] == '80104111801'].xcoord),float(sa1Centroids[sa1Centroids['SA1_CODE21'] == '80104111801'].ycoord))


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,routes,centroid,centroid_x,centroid_y,targetX,targetY,geopandas_distance
11,row-id35-shih~shjz,00000000-0000-0000-D1DC-99E0A6BDE37C,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.134996 -35.184618, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.135 -35.18462, 149.13505...",POINT (149.13703 -35.16923),149.137029,-35.169231,149.128952,-35.160728,1.465968
12,row-hnm8.89gx_jubk,00000000-0000-0000-76D4-034A416A537B,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.134996 -35.184618, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.135 -35.18462, 149.13505...",POINT (149.13697 -35.16965),149.136974,-35.169645,149.128952,-35.160728,1.507326
13,row-355y~yjis-s388,00000000-0000-0000-756A-72FFC31C62BB,0,1589437070,None,1589437070,None,{ },MULTILINESTRING ((149.12411799999998 -35.18698...,0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.12412 -35.18698, 149.124...",POINT (149.13644 -35.17102),149.136439,-35.171019,149.128952,-35.160728,1.630500
14,row-yh4w~ygqj~2mt6,00000000-0000-0000-2486-7BB7F9464708,0,1589437070,None,1589437070,None,{ },MULTILINESTRING ((149.12411799999998 -35.18698...,0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.12412 -35.18698, 149.124...",POINT (149.13644 -35.17102),149.136439,-35.171019,149.128952,-35.160728,1.630500
15,row-ytqk.np6i~ai8j,00000000-0000-0000-6CA6-B95B1861A1C2,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.134996 -35.184618, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.135 -35.18462, 149.13505...",POINT (149.13697 -35.16965),149.136974,-35.169645,149.128952,-35.160728,1.507326
16,row-mi8r~f4i3.xgsg,00000000-0000-0000-C987-418F6AC5A41B,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.134996 -35.184618, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.135 -35.18462, 149.13505...",POINT (149.13699 -35.16913),149.136985,-35.169128,149.128952,-35.160728,1.451946
17,row-hw3s~cksg_6j8n,00000000-0000-0000-02A9-275828C03DC9,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.134996 -35.184618, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.135 -35.18462, 149.13505...",POINT (149.13693 -35.16954),149.136932,-35.169543,149.128952,-35.160728,1.493327
18,row-cb6d-4hcw-p3t3,00000000-0000-0000-C067-69F0FACD6639,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.122171 -35.173339, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.12217 -35.17334, 149.122...",POINT (149.13863 -35.16737),149.138629,-35.167366,149.128952,-35.160728,1.406245
19,row-9dg4.uty4_h7v5,00000000-0000-0000-8370-2BE49E2DD338,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.143879 -35.170876, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.14388 -35.17088, 149.143...",POINT (149.13822 -35.18034),149.138216,-35.180344,149.128952,-35.160728,2.863497
20,row-9xf9~pdgt.kazj,00000000-0000-0000-34EF-3F2869D5DD37,0,1589437070,None,1589437070,None,{ },"MULTILINESTRING ((149.124645 -35.166118, 149.1...",0,...,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus,"MULTILINESTRING ((149.12464 -35.16612, 149.125...",POINT (149.1401 -35.16773),149.140104,-35.167726,149.128952,-35.160728,1.564995
